<h1>Reccomendation System</h1>

<h2>Importing Libraries</h2>

In [8]:
import numpy as np
import random as rn
import pandas as pd
import seaborn as sns
import re
import math

<h2>Importing Database</h2>

In [9]:
apps = pd.read_csv(r"googleplaystore.csv")

<h1>Data Cleanup</h1>

In [10]:
display(apps)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
10838,Parkinson Exercices FR,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


In [11]:
apps = apps.drop(columns=['Type', 'Price', 'Category','Last Updated', 'Current Ver'])
apps.head()

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,4.1,159,19M,"10,000+",Everyone,Art & Design,4.0.3 and up
1,Coloring book moana,3.9,967,14M,"500,000+",Everyone,Art & Design;Pretend Play,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",4.7,87510,8.7M,"5,000,000+",Everyone,Art & Design,4.0.3 and up
3,Sketch - Draw & Paint,4.5,215644,25M,"50,000,000+",Teen,Art & Design,4.2 and up
4,Pixel Draw - Number Art Coloring Book,4.3,967,2.8M,"100,000+",Everyone,Art & Design;Creativity,4.4 and up


In [12]:
apps['Android Ver'] = apps['Android Ver'].str.replace('[^\d.]', '')
avgVer = pd.to_numeric(apps['Android Ver'], errors='coerce').mean()
apps['Android Ver'] = apps['Android Ver'].replace('', avgVer)
apps['Android Ver'] = apps['Android Ver'].str.replace('.','a', regex=True).str.replace('a','.', 1, regex=True).str.replace('a','', regex=True)
apps['Size'] = apps['Size'].str.replace('[^\d.]', '')
apps['Installs'] = apps['Installs'].str.replace('[^\d.]', '')

avgSize = pd.to_numeric(apps['Size'], errors='coerce').mean()
apps['Size'] = apps['Size'].replace('', avgSize)
apps = apps.dropna()
apps = apps.astype({'Reviews':'float', 'Size':'float', 'Installs':'float', 'Android Ver':'float'})

C:\Users\elere\AppData\Local\Temp\ipykernel_22396\2251694235.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Android Ver'] = apps['Android Ver'].str.replace('[^\d.]', '')
C:\Users\elere\AppData\Local\Temp\ipykernel_22396\2251694235.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Size'] = apps['Size'].str.replace('[^\d.]', '')
C:\Users\elere\AppData\Local\Temp\ipykernel_22396\2251694235.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  apps['Installs'] = apps['Installs'].str.replace('[^\d.]', '')


In [13]:
class DataClean:
    @staticmethod
    def splitGenres(x):
        genresConnected = x["Genres"].unique().tolist()
        genres = set(())
        for i in genresConnected:
            afterSplit = i.split(";")
            for j in afterSplit:
                if j!='February 11, 2018':
                    genres.add(j)
        for i in list(genres):
            x[i] = [0 for x in range(x.shape[0])]
        for index in x.index:
            values = x.loc[index,'Genres'].split(";")
            for i in values:
                x.loc[index,i] = 1
        return x

In [14]:
apps = DataClean.splitGenres(apps)
apps.head()
apps.drop(columns=['Genres'], inplace=True)
apps = apps.drop_duplicates(subset='App', keep="first")

In [15]:
class DataProcessing:
    @staticmethod
    def shuffle(x):
        for i in range(len(x)-1,0,-1):
            j=rn.randint(0,i-1)
            x.iloc[j], x.iloc[i] = x.iloc[i], x.iloc[j]
            
    @staticmethod
    def normalization(x):
        values=x.select_dtypes(exclude="object")
        columnNames=values.columns.tolist()
        for column in columnNames[:-1]:
            data = x.loc[:,column]
            max1=max(data)
            min1=min(data)
            for row in range(0,len(x),1):
                xprim=(x.at[row, column]-min1)/(max1-min1)
                x.at[row, column]=xprim
                
    @staticmethod
    def split(x,k):
        return x[:int(len(x)*k//1)], x[int(len(x)*k//1 +1):]

In [16]:
DataProcessing.shuffle(apps)
apps = apps.reset_index(drop=True)

In [17]:
DataProcessing.normalization(apps)

In [19]:
trainingSet, validatingSet = DataProcessing.split(apps, 0.999)
validatingSet = validatingSet.reset_index(drop=True)
validatingSet

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Casual,Health & Fitness,...,Medical,Finance,Shopping,Music & Video,Books & Reference,Productivity,Action,Strategy,Libraries & Demo,Travel & Local
0,Subway Simulator 3D,0.750,5.677661e-04,0.061430,9.999990e-04,Everyone,Simulation,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BG Remover & Eraser Pro,0.300,6.682747e-08,0.003827,4.990000e-07,Everyone,Tools,0.185714,0,0,...,0,0,0,0,0,0,0,0,0,0
2,swissinfo.ch,0.725,6.014472e-06,0.008661,4.999900e-05,Everyone 10+,News & Magazines,0.485714,0,0,...,0,0,0,0,0,0,0,0,0,0
3,photo keypad lockscreen,0.775,5.024156e-03,0.007150,9.999999e-03,Everyone,Tools,0.428571,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CW Beacon for Ham Radio,0.950,2.004824e-07,0.518630,9.900000e-08,Everyone,Communication,0.085714,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Cookpad - FREE recipe search makes fun cooking...,0.775,1.443095e-03,0.012085,9.999999e-03,Everyone,Food & Drink,0.571429,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Baby Monitor AV,0.575,3.192125e-05,0.008560,9.999900e-05,Everyone,Communication,0.185714,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
apps["Content Rating"].unique().tolist()

['Everyone',
 'Teen',
 'Everyone 10+',
 'Mature 17+',
 'Unrated',
 'Adults only 18+']

In [21]:
if 'Mature 17+' in validatingSet["Content Rating"].unique().tolist() or 'Adults only 18+' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Adults only 18+' for i in range(len(validatingSet))]
elif 'Teen' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Teen' for i in range(len(validatingSet))]

In [22]:
validatingSet.sort_values(by=['Content Rating'])

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Casual,Health & Fitness,...,Medical,Finance,Shopping,Music & Video,Books & Reference,Productivity,Action,Strategy,Libraries & Demo,Travel & Local
0,Subway Simulator 3D,0.750,5.677661e-04,0.061430,9.999990e-04,Everyone,Simulation,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BG Remover & Eraser Pro,0.300,6.682747e-08,0.003827,4.990000e-07,Everyone,Tools,0.185714,0,0,...,0,0,0,0,0,0,0,0,0,0
3,photo keypad lockscreen,0.775,5.024156e-03,0.007150,9.999999e-03,Everyone,Tools,0.428571,0,0,...,0,0,0,0,0,0,0,0,0,0
4,CW Beacon for Ham Radio,0.950,2.004824e-07,0.518630,9.900000e-08,Everyone,Communication,0.085714,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Cookpad - FREE recipe search makes fun cooking...,0.775,1.443095e-03,0.012085,9.999999e-03,Everyone,Food & Drink,0.571429,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Baby Monitor AV,0.575,3.192125e-05,0.008560,9.999900e-05,Everyone,Communication,0.185714,0,0,...,0,0,0,0,0,0,0,0,0,0
2,swissinfo.ch,0.725,6.014472e-06,0.008661,4.999900e-05,Everyone 10+,News & Magazines,0.485714,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
trainingSet.loc[trainingSet['App'] == 'BURGER KING® App']

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Casual,Health & Fitness,...,Medical,Finance,Shopping,Music & Video,Books & Reference,Productivity,Action,Strategy,Libraries & Demo,Travel & Local
6929,BURGER KING® App,0.75,0.00093,0.038268,0.005,Everyone,Lifestyle,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,0


<h2>k-nearest neighbors algorithm</h2>

In [24]:
class KNN:
    @staticmethod
    def distance(v1, v2):
        sum = 0
        for j in range(1,len(v1)):
            if j != 5 and j!=6 and j!=7:
                sum += (v1[j] - v2[j])**2
        return sum**(1/2)
    def clustering(dataset, sample):
        distances = []
        for i in range(len(dataset)):
            distances.append(KNN.distance(sample, dataset.iloc[i]))
            if dataset.iloc[i]['Android Ver'] > sample['Android Ver']:
                distances[i] = 10
            if sample['Content Rating'] != 'Adults only 18+':
                if dataset.iloc[i]['Content Rating'] in ['Mature 17+','Adults only 18+']:
                    distances[i] = 10
            elif sample['Content Rating'] != 'Teen':
                if dataset.iloc[i]['Content Rating'] in ['Teen','Mature 17+','Adults only 18+']:
                    distances[i] = 10
        dataset["distance"] = distances
                    
        return dataset
    
    def recommendation(dataset,samples):
        sorted = []
        dataset["distance"] = [10 for x in range(len(dataset))]
        for index in range(len(samples)):
            sorted.append(KNN.clustering(dataset.copy(),samples.iloc[index]))
        distances = []
        for index in dataset.index:
#             dataset.loc[index,'distance']
            distance = []
            for i in range(len(sorted)):
                distance.append(sorted[i].loc[index,'distance'])
            avg = []
            for i in range(3):
                avg.append(min(distance))
                distance.remove(min(distance))
            distances.append(sum(avg)/3)
        dataset["distance"] = distances
        dataset = dataset.sort_values("distance")
        return dataset.head(10)

<h2>Apps Recommendation</h2>

<h4>Method 1</h4>

In [25]:
avgRating = pd.to_numeric(validatingSet['Rating'], errors='coerce').mean()
avgReviews = pd.to_numeric(validatingSet['Reviews'], errors='coerce').mean()
avgSize = pd.to_numeric(validatingSet['Size'], errors='coerce').mean()
avgInstalls = pd.to_numeric(validatingSet['Installs'], errors='coerce').mean()
maxAndroidVer = max(validatingSet['Android Ver'])
genres = []
for i in validatingSet.columns.tolist()[8:]:
    genres.append(pd.to_numeric(validatingSet[i], errors='coerce').mean())
avgAppList = ['avgApp',avgRating,avgReviews,avgSize,avgInstalls,validatingSet.iloc[0]['Content Rating'],"avgGenre",maxAndroidVer]
for i in genres:
    avgAppList.append(i)
avgApp = pd.DataFrame(columns=validatingSet.columns.tolist())
avgApp.loc[0] = avgAppList
recommendedApps = KNN.clustering(trainingSet.copy(), avgApp.iloc[0])
recommendedApps.sort_values("distance").head()

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Casual,Health & Fitness,...,Finance,Shopping,Music & Video,Books & Reference,Productivity,Action,Strategy,Libraries & Demo,Travel & Local,distance
2117,AH Alarm Panel,0.725,1.336549e-07,0.080564,9.900000e-08,Everyone,Tools,0.071429,0,0,...,0,0,0,0,0,0,0,0,0,0.808807
2047,My Vodacom SA,0.675,5.573411e-04,0.060423,4.999999e-03,Everyone,Communication,0.471429,0,0,...,0,0,0,0,0,0,0,0,0,0.808814
5717,[root] Pry-Fi,0.725,1.500277e-04,0.071501,4.999990e-04,Everyone,Tools,0.457143,0,0,...,0,0,0,0,0,0,0,0,0,0.808947
6937,WiFi Access Point (hotspot),0.725,1.521439e-05,0.060423,9.999900e-05,Everyone,Communication,0.185714,0,0,...,0,0,0,0,0,0,0,0,0,0.809258
6621,TownWiFi | Wi-Fi Everywhere,0.725,5.281597e-05,0.057402,4.999990e-04,Everyone,Communication,0.457143,0,0,...,0,0,0,0,0,0,0,0,0,0.809368


<h4>Method 2</h4>

In [26]:
KNN.recommendation(trainingSet.copy(),validatingSet.copy())

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Casual,Health & Fitness,...,Finance,Shopping,Music & Video,Books & Reference,Productivity,Action,Strategy,Libraries & Demo,Travel & Local,distance
3834,Smart Air Conditioner,0.575,1.065898e-04,0.012085,0.00050,Everyone,Tools,0.171429,0,0,...,0,0,0,0,0,0,0,0,0,0.629896
6976,Smart Air Conditioner(CAC),0.550,9.088535e-06,0.006949,0.00005,Everyone,Tools,0.171429,0,0,...,0,0,0,0,0,0,0,0,0,0.629910
1082,AZ PLAYER HD,0.550,5.123439e-07,0.003726,0.00001,Everyone,Tools,0.171429,0,0,...,0,0,0,0,0,0,0,0,0,0.629916
2148,SportCAM,0.550,9.489500e-06,0.008560,0.00010,Everyone,Tools,0.171429,0,0,...,0,0,0,0,0,0,0,0,0,0.629919
1481,Air conditioner remote control,0.600,6.650001e-04,0.004028,0.00100,Everyone,Tools,0.185714,0,0,...,0,0,0,0,0,0,0,0,0,0.629919
2982,DW VMAX,0.600,1.875624e-05,0.007553,0.00010,Everyone,Tools,0.171429,0,0,...,0,0,0,0,0,0,0,0,0,0.629937
6288,"Ruler(cm, inch)",0.600,6.433257e-05,0.001511,0.00050,Everyone,Tools,0.185714,0,0,...,0,0,0,0,0,0,0,0,0,0.629960
1971,Download Manager,0.700,1.762708e-03,0.001208,0.01000,Everyone,Tools,0.185714,0,0,...,0,0,0,0,0,0,0,0,0,0.629978
1165,Don't touch my phone,0.700,4.887761e-04,0.001309,0.00500,Everyone,Tools,0.185714,0,0,...,0,0,0,0,0,0,0,0,0,0.630010
4283,Graphing Calculator,0.725,2.759306e-04,0.004330,0.00100,Everyone,Tools,0.185714,0,0,...,0,0,0,0,0,0,0,0,0,0.630108


<h2>Importing Reviews Database</h2>

In [27]:
reviews = pd.read_csv(r"processedReviews.csv")

In [28]:
display(reviews)

,App,reviews
0,10 Best Foods for You,best;food;health;healthy;love
1,104 找工作 - 找工作 找打工 找兼職 履歷健檢 履歷診療室,easy;job;use;useful;mobile
2,11st,for;english;there;language;if
3,1800 Contacts - Lens Store,easy;contacts;order;super;love
4,1LINE – One Line with One Touch,game;play;love;ads;many
...,...,...
860,Hotels.com: Book Hotel Rooms & Find Vacation D...,hotel;book;pet;friendly;find
861,Hotspot Shield Free VPN Proxy & Wi-Fi Security,trial;days;free;asking;credit
862,Hotstar,shows;answer;time;u;playing
863,Hotwire Hotel & Car Rental App,would;hotels;hotel;better;card


In [29]:
appsWithReviews = pd.merge(apps, reviews, on="App")

In [30]:
display(appsWithReviews)

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Casual,Health & Fitness,...,Finance,Shopping,Music & Video,Books & Reference,Productivity,Action,Strategy,Libraries & Demo,Travel & Local,reviews
0,Avakin Life - 3D virtual world,0.850,0.028410,0.090634,0.0100,Teen,Role Playing,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,game;get;love;money;would
1,"2Date Dating App, Love and matching",0.850,0.000927,0.007150,0.0005,Mature 17+,Dating,0.432857,0,0,...,0,0,0,0,0,0,0,0,0,users;dating;members;vip;send
2,FlipaClip - Cartoon animation,0.825,0.004326,0.038268,0.0050,Everyone,Art & Design,0.432857,0,0,...,0,0,0,0,0,0,0,0,0,im;make;pmv;animation;give
3,Build a Bridge!,0.900,0.005869,0.073515,0.0100,Everyone,Puzzle,0.442857,0,0,...,0,0,0,0,0,0,0,0,0,game;play;level;controls;fun
4,Hero Hunters,0.850,0.001576,0.098691,0.0050,Teen,Action,0.485714,0,0,...,0,0,0,0,0,1,0,0,0,game;pvp;play;still;gold
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
617,Bowmasters,0.925,0.034181,0.036395,0.0500,Teen,Action,0.442857,0,0,...,0,0,0,0,0,1,0,0,0,game;ads;fun;get;characters
618,BZWBK24 mobile,0.875,0.001447,0.038268,0.0010,Everyone,Finance,0.442857,0,0,...,1,0,0,0,0,0,0,0,0,in;application;of;only;will
619,Beautiful Widgets Pro,0.800,0.002181,0.013092,0.0010,Everyone,Personalization,0.185714,0,0,...,0,0,0,0,0,0,0,0,0,time;weather;widget;update;years
620,Google Duo - High Quality Video Calls,0.900,0.046406,0.036395,0.5000,Everyone,Communication,0.485714,0,0,...,0,0,0,0,0,0,0,0,0,video;duo;calls;google;able


In [31]:
trainingSet, validatingSet = DataProcessing.split(appsWithReviews, 0.995)
validatingSet = validatingSet.reset_index(drop=True)
validatingSet

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Casual,Health & Fitness,...,Finance,Shopping,Music & Video,Books & Reference,Productivity,Action,Strategy,Libraries & Demo,Travel & Local,reviews
0,Beautiful Widgets Pro,0.80,0.002181,0.013092,0.001,Everyone,Personalization,0.185714,0,0,...,0,0,0,0,0,0,0,0,0,time;weather;widget;update;years
1,Google Duo - High Quality Video Calls,0.90,0.046406,0.036395,0.500,Everyone,Communication,0.485714,0,0,...,0,0,0,0,0,0,0,0,0,video;duo;calls;google;able
2,Amazon for Tablets,0.75,0.003155,0.021148,0.010,Teen,Shopping,0.485714,0,0,...,0,1,0,0,0,0,0,0,0,update;australian;store;amazon;still


In [32]:
if 'Mature 17+' in validatingSet["Content Rating"].unique().tolist() or 'Adults only 18+' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Adults only 18+' for i in range(len(validatingSet))]
elif 'Teen' in validatingSet["Content Rating"].unique().tolist():
    validatingSet["Content Rating"] = ['Teen' for i in range(len(validatingSet))]

In [33]:
class KNNWithJaccardSimilarityCoefficient:
    @staticmethod
    def distance(v1, v2):
        sum = 0
        for j in range(1,len(v1)-1):
            if j != 5 and j!=6 and j!=7:
                sum += (v1[j] - v2[j])**2
        return (sum**(1/2)) * KNNWithJaccardSimilarityCoefficient.jaccard(v1[len(v1)-1], v2[len(v1)-1])
    
    def jaccard(string1,string2):
        sim = 0
        try:
            string1 = string1.split(";")
        except:
            string1 = [""]       
        try:
            string2 = string2.split(";")
        except:
            string2 = [""]
            
        for word in string1:
            if word in string2:
                sim+=1
        return 1 - (sim/5)
    
    def clustering(dataset, sample):
        distances = []
        for i in range(len(dataset)):
            distances.append(KNNWithJaccardSimilarityCoefficient.distance(sample, dataset.iloc[i]))
            if dataset.iloc[i]['Android Ver'] > sample['Android Ver']:
                distances[i] = 10
            if sample['Content Rating'] != 'Adults only 18+':
                if dataset.iloc[i]['Content Rating'] in ['Mature 17+','Adults only 18+']:
                    distances[i] = 10
            elif sample['Content Rating'] != 'Teen':
                if dataset.iloc[i]['Content Rating'] in ['Teen','Mature 17+','Adults only 18+']:
                    distances[i] = 10
        dataset["distance"] = distances
                    
        return dataset
    
    def recommendation(dataset,samples):
        sorted = []
        dataset["distance"] = [10 for x in range(len(dataset))]
        for index in range(len(samples)):
            sorted.append(KNNWithJaccardSimilarityCoefficient.clustering(dataset.copy(),samples.iloc[index]))
        distances = []
        for index in dataset.index:
#             dataset.loc[index,'distance']
            distance = []
            for i in range(len(sorted)):
                distance.append(sorted[i].loc[index,'distance'])
            avg = []
            for i in range(3):
                avg.append(min(distance))
                distance.remove(min(distance))
            distances.append(sum(avg)/3)
        dataset["distance"] = distances
        dataset = dataset.sort_values("distance")
        return dataset.head(10)

<h4>Method 1</h4>

In [34]:
avgRating = pd.to_numeric(validatingSet['Rating'], errors='coerce').mean()
avgReviews = pd.to_numeric(validatingSet['Reviews'], errors='coerce').mean()
avgSize = pd.to_numeric(validatingSet['Size'], errors='coerce').mean()
avgInstalls = pd.to_numeric(validatingSet['Installs'], errors='coerce').mean()
maxAndroidVer = max(validatingSet['Android Ver'])
genres = []
for i in validatingSet.columns.tolist()[8:-1]:
    genres.append(pd.to_numeric(validatingSet[i], errors='coerce').mean())
avgAppList = ['avgApp',avgRating,avgReviews,avgSize,avgInstalls,validatingSet.iloc[0]['Content Rating'],"avgGenre",maxAndroidVer]
for i in genres:
    avgAppList.append(i)
revs = []
for i in validatingSet["reviews"].tolist():
    i = i.split(";")
    for j in i:
        revs.append(j)
words = {}
for word in revs:
        words[word] = 0
for word in revs:
        words[word] +=1
revs = []
for i in range(5):
    try:
        revs.append(max(words, key=words.get))
        del words[max(words, key=words.get)]
    except:
        pass
avgWords = ";".join(revs)
avgAppList.append(avgWords)

avgApp = pd.DataFrame(columns=validatingSet.columns.tolist())
avgApp.loc[0] = avgAppList
recommendedApps = KNNWithJaccardSimilarityCoefficient.clustering(trainingSet.copy(), avgApp.iloc[0])
recommendedApps.sort_values("distance").head(10)

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Casual,Health & Fitness,...,Shopping,Music & Video,Books & Reference,Productivity,Action,Strategy,Libraries & Demo,Travel & Local,reviews,distance
558,Beautiful Widgets Free,0.725,0.000558,0.013092,0.0010,Everyone,Personalization,0.185714,0,0,...,0,0,0,0,0,0,0,0,weather;location;widget;time;update,0.167825
33,Fancy Widgets,0.800,0.000829,0.002920,0.0050,Everyone,Personalization,0.085714,0,0,...,0,0,0,0,0,0,0,0,love;widget;clock;weather;time,0.333461
201,HD Widgets,0.825,0.001306,0.025176,0.0010,Everyone,Personalization,0.485714,0,0,...,0,0,0,0,0,0,0,0,weather;widget;time;widgets;clock,0.333627
552,DU Browser—Browse fast & fun,0.825,0.025250,0.003726,0.0100,Everyone,Communication,0.428571,0,0,...,0,0,0,0,0,0,0,0,engine;video;browser;cant;time,0.665925
535,Family Dollar,0.825,0.000058,0.020141,0.0010,Everyone,Shopping,0.442857,0,0,...,1,0,0,0,0,0,0,0,time;store;coupons;dollar;coupon,0.667277
91,Gyft - Mobile Gift Card Wallet,0.775,0.000216,0.013092,0.0005,Everyone,Shopping,0.485714,0,0,...,1,0,0,0,0,0,0,0,card;gift;cards;balance;time,0.668201
435,Find&Save - Local Shopping,0.750,0.000102,0.005237,0.0005,Everyone,Shopping,0.442857,0,0,...,1,0,0,0,0,0,0,0,get;cash;time;back;receipt,0.669607
329,Digital Alarm Clock,0.800,0.002638,0.015106,0.0100,Everyone,Tools,0.485714,0,0,...,0,0,0,0,0,0,0,0,alarm;clock;update;time;work,0.699612
97,FindShip,0.800,0.000519,0.007553,0.0010,Everyone,Maps & Navigation,0.442857,0,0,...,0,0,0,0,0,0,0,0,ship;update;time;ships;see,0.700439
502,ForecaWeather,0.800,0.000410,0.004330,0.0010,Everyone,Weather,0.442857,0,0,...,0,0,0,0,0,0,0,0,widget;weather;location;accurate;data,0.700470


<h4>Method 2</h4>

In [35]:
KNNWithJaccardSimilarityCoefficient.recommendation(trainingSet.copy(),validatingSet.copy())

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Casual,Health & Fitness,...,Shopping,Music & Video,Books & Reference,Productivity,Action,Strategy,Libraries & Demo,Travel & Local,reviews,distance
558,Beautiful Widgets Free,0.725,0.000558,0.013092,0.00100,Everyone,Personalization,0.185714,0,0,...,0,0,0,0,0,0,0,0,weather;location;widget;time;update,0.885771
33,Fancy Widgets,0.800,0.000829,0.002920,0.00500,Everyone,Personalization,0.085714,0,0,...,0,0,0,0,0,0,0,0,love;widget;clock;weather;time,0.974123
612,BIG Launcher,0.825,0.000020,0.004935,0.00001,Everyone,Personalization,0.157143,0,0,...,0,0,0,0,0,0,0,0,launcher;screen;phone;big;buttons,0.981884
290,ExDialer - Dialer & Contacts,0.800,0.002790,0.001712,0.01000,Everyone,Communication,0.157143,0,0,...,0,0,0,0,0,0,0,0,dialer;call;phone;stock;contacts,1.110905
573,Caller ID +,0.750,0.000212,0.117825,0.00100,Everyone,Communication,0.185714,0,0,...,0,0,0,0,0,0,0,0,found;what;scam;havent;even,1.121972
143,GO Notifier,0.800,0.002770,0.698892,0.01000,Everyone,Communication,0.142857,0,0,...,0,0,0,0,0,0,0,0,go;notification;work;notifier;messages,1.323995
607,Block Puzzle,0.900,0.001332,0.006848,0.00500,Everyone,Puzzle,0.185714,0,0,...,0,0,0,0,0,0,0,0,game;time;fun;play;love,1.351916
571,Galactic Core Free Wallpaper,0.775,0.001546,0.858006,0.01000,Everyone,Personalization,0.157143,0,0,...,0,0,0,0,0,0,0,0,wallpaper;version;get;even;live,1.400443
250,Anger of stick 5 : zombie,0.875,0.012230,0.022155,0.05000,Teen,Action,0.185714,0,0,...,0,0,0,0,1,0,0,0,game;cool;keeps;engaged;keep,1.440729
600,AppLock - Fingerprint,0.850,0.016601,0.002518,0.01000,Everyone,Tools,0.185714,0,0,...,0,0,0,0,0,0,0,0,lock;phone;working;password;please,1.443794


<h2>KD Tree Algorithm</h2>

In [36]:
class Node:
    def __init__(self, point, left=None, right=None, name=None):
        self.point = point
        self.left = left
        self.right = right
        self.name = name


class KDTree:
    def __init__(self, points):
        self.k = 4  # Number of dimensions
        self.root = self.build_tree(points)

    def build_tree(self, points, depth=0):
        if len(points) == 0:
            return None

        # Select axis based on depth
        axis = depth % self.k

        # Sort points based on the axis
        sorted_points = points.sort_values(by=points.columns[axis+2])

        # Find the median
        median_index = len(sorted_points) // 2
        median = sorted_points.iloc[median_index]
        
        # Recursively build subtrees
        left_points = sorted_points[:median_index]
        right_points = sorted_points[median_index + 1:]

        left = self.build_tree(left_points, depth + 1)
        right = self.build_tree(right_points, depth + 1)
        return Node(median[1:5], left, right, median[0])

    def find_nearest_neighbors(self, query_point, k=1):
        nearest_points = []
        distances = []
        nearest_names = []

        def search(node, depth=0):
            if node is None:
                return

            # Select axis based on depth
            axis = depth % self.k

            # Traverse left or right subtree based on query point
            if query_point[axis] < node.point[axis]:
                search(node.left, depth + 1)
            else:
                search(node.right, depth + 1)
            # Calculate distance between query point and current point
            distance = np.linalg.norm(np.array(query_point) - np.array(node.point))
            # If the number of nearest points is less than k, add current point
            if len(nearest_points) == 0:
                distances.append(distance)
                nearest_points.append(node.point)
                nearest_names.append(node.name)
            
            elif len(nearest_points) < k:
                index = len(nearest_points)
                for i in range(0, len(nearest_points)):
                    if(distance < distances[i]):
                        index = i
                        break
                distances.insert(index, distance)
                nearest_points.insert(index, node.point)
                nearest_names.insert(index, node.name)
            else:
                # If the current point is closer than the farthest nearest point,
                # replace the farthest point with the current point
                if distance < distances[-1]:
                    for i in range(0, k):
                        if(distance < distances[i]):
                            distances.insert(i, distance)
                            nearest_points.insert(i, node.point)
                            nearest_names.insert(i, node.name)
                            break
                    distances.pop()
                    nearest_points.pop()
                    nearest_names.pop()
            # Check if there can be closer points on the other side of the splitting axis
            if abs(query_point[axis] - node.point[axis]) < distances[-1] or len(nearest_points) < k:
                if query_point[axis] < node.point[axis]:
                    search(node.right, depth + 1)
                else:
                    search(node.left, depth + 1)
        search(self.root)
        return nearest_names
    

<h2>Divide apps dataset by genres</h2>

In [68]:
def divide_by_genres(dataset):
    datasets = []
    for i in range(8, dataset.shape[1]):
        datasets.append(dataset[dataset.iloc[:, i] == 1])
    return datasets
trainingSet_genres = divide_by_genres(trainingSet)
trainingSet_genres[52].head()
    

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Casual,Health & Fitness,...,Finance,Shopping,Music & Video,Books & Reference,Productivity,Action,Strategy,Libraries & Demo,Travel & Local,reviews
24,Amtrak,0.675,0.000375,0.027190,0.0010,Everyone,Travel & Local,0.571429,0,0,...,0,0,0,0,0,0,0,0,1,train;ticket;number;reservation;even
28,GPS Status & Toolbox,0.875,0.003335,0.003122,0.0100,Everyone,Travel & Local,0.428571,0,0,...,0,0,0,0,0,0,0,0,1,gps;best;useful;accurate;well
56,"Expedia Hotels, Flights & Car Rental Travel Deals",0.775,0.003043,0.013092,0.0100,Everyone,Travel & Local,0.571429,0,0,...,0,0,0,0,0,0,0,0,1,expedia;hotels;booking;hotel;get
66,Cheap Flights & Hotels momondo,0.800,0.000948,0.023162,0.0050,Everyone,Travel & Local,0.485714,0,0,...,0,0,0,0,0,0,0,0,1,price;flights;option;momondo;flight
100,Ascape VR: 360° Virtual Travel,0.775,0.000020,0.006647,0.0001,Everyone,Travel & Local;Action & Adventure,0.485714,0,0,...,0,0,0,0,0,0,0,0,1,paid;even;data;download;tours


<h2>Building the KD Tree</h2>

In [104]:
validatingSet

,App,Rating,Reviews,Size,Installs,Content Rating,Genres,Android Ver,Casual,Health & Fitness,...,Finance,Shopping,Music & Video,Books & Reference,Productivity,Action,Strategy,Libraries & Demo,Travel & Local,reviews
0,Beautiful Widgets Pro,0.80,0.002181,0.013092,0.001,Teen,Personalization,0.185714,0,0,...,0,0,0,0,0,0,0,0,0,time;weather;widget;update;years
1,Google Duo - High Quality Video Calls,0.90,0.046406,0.036395,0.500,Teen,Communication,0.485714,0,0,...,0,0,0,0,0,0,0,0,0,video;duo;calls;google;able
2,Amazon for Tablets,0.75,0.003155,0.021148,0.010,Teen,Shopping,0.485714,0,0,...,0,1,0,0,0,0,0,0,0,update;australian;store;amazon;still


In [112]:
kdtrees = []
for i in range(len(trainingSet_genres)):
    kdtrees.append(KDTree(trainingSet_genres[i]))
def recommendation_genres(dataset, app):
    app_genres = []
    recommended_apps = []
    for i in range(8, dataset.shape[1]):
        if app[i] == 1:
            app_genres.append(i-8)
    for i in range(len(app_genres)):
        recommended_apps_genre = kdtrees[app_genres[i]].find_nearest_neighbors(app[1:5].to_numpy(), 5)
        recommended_apps.extend(x for x in recommended_apps_genre if x not in recommended_apps)
    return recommended_apps
def kdtree_recommendation(dataset, apps):
    recommended_apps = []
    for idx, row in apps.iterrows():
        recommended_apps.extend(x for x in recommendation_genres(dataset, row) if x not in recommended_apps)
    distances = []
    for idx, i in enumerate(recommended_apps):
        distances.append(0)
        for _, j in apps.iterrows():
            distances[idx] += (np.linalg.norm(np.array(dataset[dataset["App"] == i].iloc[0][1:5]) - np.array(j[1:5])))
    best_apps = []
    for i in range(5):
        best_apps.append(recommended_apps[distances.index(min(distances))])
        distances.remove(min(distances))
        recommended_apps.remove(best_apps[-1])
    return best_apps

kdtree_recommendation(trainingSet, validatingSet)

['Gyft - Mobile Gift Card Wallet',
 'CheckPoints 🏆 Rewards App',
 'Horses Live Wallpaper',
 'Fancy Widgets',
 'HauteLook']

In [82]:
#average app
query_point = avgApp[['Rating', 'Reviews', 'Size', 'Installs']].to_numpy()[0]
print("Query point:", query_point)
nearest_neighbors = kdtree.find_nearest_neighbors(query_point, k=5)

print("Nearest neighbors:")
for neighbor in nearest_neighbors:
    print(neighbor)

Query point: [0.81666667 0.01724696 0.02354506 0.17033333]
Nearest neighbors:
Amazon Shopping
Amazon Prime Video
Akinator
Anger of stick 5 : zombie
GUNSHIP BATTLE: Helicopter 3D


In [39]:
apps2 = pd.read_csv(r"googleplaystore.csv")
len(apps2['Genres'].unique().tolist())
print(len(apps))

7249
